<a href="https://colab.research.google.com/github/KotaroT1/KotaroT1/blob/main/Diabetes_Prediction_Challenge_Kaggle_2025_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Google Driveへのマウント
# 実行するとGoogleログイン画面が出るため、コンペデータを格納しているアカウントでログインしてください

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 必要モジュールでColab環境にないものはinstall
!pip install japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 54.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=875af15066dd56feca3b506165e0543c896dc2c422403665ea162498fb6c9f5d
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize_matplotlib


In [9]:
# 必要モジュールをimport
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns
import japanize_matplotlib
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.metrics import make_scorer
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ランダム性を伴う処理を行うため、結果の再現性を保つにはシード値を固定しておく必要があります
SEED = 42
np.random.seed(SEED)

In [10]:
# Google Driveがマウントされているか確認し、base_pathを設定
if os.path.exists('/content/drive/My Drive'):
    # Google Driveがマウントされている場合
    # ご自身の環境に合わせて下記を適切なパスに変更してください
    base_path = '/content/drive/MyDrive/Colab Notebooks/Kaggle_DiabetesPredictionChallenge/data'
else:
    # Google Driveがマウントされていない場合
    base_path = './'

print(f"データ読み込み元パス: {base_path}")


# データ読み込み
# player_idやmatch_idの数値的大小に意味はないのでstring形式で読み込み
train_df = pd.read_csv(f"{base_path}/train.csv")
test_df = pd.read_csv(f"{base_path}/test.csv")
submission_df = pd.read_csv(f"{base_path}/sample_submission.csv")

print(f"trainデータ形状: {train_df.shape}")
display(train_df.head(3))
print(f"\ntestデータ形状: {test_df.shape}")
display(test_df.head(3))

データ読み込み元パス: /content/drive/MyDrive/Colab Notebooks/Kaggle_DiabetesPredictionChallenge/data
trainデータ形状: (700000, 26)


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0



testデータ形状: (300000, 25)


,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,...,111,Female,White,Highschool,Middle,Former,Employed,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,...,145,Female,White,Highschool,Middle,Never,Unemployed,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,...,184,Male,White,Highschool,Low,Never,Employed,0,0,0


In [11]:
# Inspect data
print("Train Head:")
print(train_df.head())
print("\nTrain Info:")
print(train_df.info())
print("\nTest Head:")
print(test_df.head())
print("\nTest Info:")
print(test_df.info())

# Check target distribution
print("\nTarget Distribution:")
print(train_df['diagnosed_diabetes'].value_counts(normalize=True))

Train Head:
   id  age  alcohol_consumption_per_week  physical_activity_minutes_per_week  \
0   0   31                             1                                  45   
1   1   50                             2                                  73   
2   2   32                             3                                 158   
3   3   54                             3                                  77   
4   4   54                             1                                  55   

   diet_score  sleep_hours_per_day  screen_time_hours_per_day   bmi  \
0         7.7                  6.8                        6.1  33.4   
1         5.7                  6.5                        5.8  23.8   
2         8.5                  7.4                        9.1  24.1   
3         4.6                  7.0                        9.2  26.6   
4         5.7                  6.2                        5.1  28.8   

   waist_to_hip_ratio  systolic_bp  ...  gender  ethnicity  education_level  \
0

In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [14]:
# Preprocessing
X = train_df.drop(['id', 'diagnosed_diabetes'], axis=1)
y = train_df['diagnosed_diabetes']
X_test = test_df.drop(['id'], axis=1)
test_ids = test_df['id']

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(exclude=['object']).columns.tolist()

print(f"Categorical columns: {categorical_cols}")

# Encode categorical variables
# Use OrdinalEncoder. Handle unknown values just in case (though not expected given the clean data).
preprocessor = make_column_transformer(
    (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols),
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform="pandas")

X_encoded = preprocessor.fit_transform(X)
X_test_encoded = preprocessor.transform(X_test)

# Boolean mask for categorical features
# After transformation, categorical columns come first because of make_column_transformer order
cat_mask = [col in categorical_cols for col in X_encoded.columns]

# Train model
print("Training HistGradientBoostingClassifier...")
model = HistGradientBoostingClassifier(
    loss='log_loss',
    learning_rate=0.05,
    max_iter=500,
    max_leaf_nodes=31,
    categorical_features=cat_mask,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10
)

model.fit(X_encoded, y)

# Predict
print("Predicting...")
probs = model.predict_proba(X_test_encoded)[:, 1]

# Create submission
submission = pd.DataFrame({
    'id': test_ids,
    'diagnosed_diabetes': probs
})

# Save to CSV
submission_filename = 'submission.csv'
submission.to_csv(submission_filename, index=False)

print(f"Submission file created: {submission_filename}")
print(submission.head())

Categorical columns: ['gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status']
Training HistGradientBoostingClassifier...
Predicting...
Submission file created: submission.csv
       id  diagnosed_diabetes
0  700000            0.506004
1  700001            0.690161
2  700002            0.774031
3  700003            0.415432
4  700004            0.913394


In [15]:
submission.to_csv(f"{base_path}/DPC_submissions_02.csv", index=False)